In [1]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import distance

### Jupyter's trick

Tab: Compelete  
Shift + Tab Parameter: Quick inspect the parameters  
Shift + Tab X2 or X3: More information  
??function: Sourcode  
?function: Documentation  

### Theorems, Lingos

_Curse of dimensionality_: The more columns we have, the more empty space it creates. KNN works very well in high dimensions despite what the theory said because the points still have different distances away from each other.

_No free lunch theorem_: In theory, there is no model that works well for any kind of dataset. In practice, we can use random forest for nearly all kind of dataset.

_Churn_: Loss of users

_Viral coefficient_: the ability of a business to tap into an early user base and get that user base to tell other people about it. If a site has a viral coefficient of 10%, and has 1,000 users, then after one month it will gain another 100 users through viral channels.

_Ground truth_: Ground truth refers to strong labels that have a high likelihood of being accurate, knowledge that we're very sure is True.

_Correlation does not imply causation!_: Just because we can see a connection or a mutual relationship between two variables, it doesn't mean that one causes the other.

_A/B Testing_

### Tips

_Regression_: Predict continous variables  
_Classify_: Predict categorical variables

If the dataset contain a mix of continuous and categorical variables, convert everything into numbers  
Always do feature extraction step when working with date-time, so we can capture any trend/cyclical behavior like holiday, weekend, sport event, rainning that day, etc. df.saledate.__dt.

In [2]:
# Save stuff after we process so we can accessed efficiently
# os.makedirs('tmp', exist_ok=True)
# df.to_feather('tmp/raw')

# In the future we can simply read it from this last format:
# df = pd.read_feather('tmp/raw')

r^2 score: Ratio between how good my model is VS how good is the naive mean model

Use _ variable if we want to throw something away

### Random Forest

Build a decision tree from scratch:  
    Calculate the root mean squared error of the 1st split. This number would represents how good a split is.
    Try all variables and all possible value of that variable and see which variable and which value gives us a split with the best score.
    Stop splitting when the leaf node only has 1 thing in it.

Random Forest (Predict on uncorrelated trees):  
    Grab some rows as random then put them into a smaller dataset and build a tree based on that
    Do it again with a different random subset
    Make prediction on each tree
    Take an average  

Use Out-of-bag (OOB) error to prevent over fitting or for small dataset: Pass un-used rows to the 1st tree and treat it as a validation set. Do the same thing for a 2nd tree. To calculation prediction, we would average all the trees where that row is not used for training

In [ ]:
Use Grid Search